# 사전학습 모델 사용하여 <br>리그오브레전드 소환사 이름으로 티어 예측하기

## 필요 라이브러리 설치 및 불러오기

In [1]:
# 한글 자모 단위 처리 패키지 설치
!pip install hgtk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for hgtk: filename=hgtk-0.1.3-py2.py3-none-any.whl size=6689 sha256=813fbded7b0a3f3cf82f3b2a618eb771d53636c28441ca79416e323b1310417f
  Stored in directory: /root/.cache/pip/wheels/a6/e9/bc/524beb5222b11aa439a23a07be5bd8a559d266153103c37979
Successfully built hgtk


In [18]:
import hgtk
import numpy as np
import urllib
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
def word_to_jamo(token):
  def to_special_token(jamo):
    if not jamo:
      return '-'
    else:
      return jamo

  decomposed_token = ''
  for char in token:
    try:
      # char(음절)을 초성, 중성, 종성으로 분리
      cho, jung, jong = hgtk.letter.decompose(char)

      # 자모가 빈 문자일 경우 특수문자 -로 대체
      cho = to_special_token(cho)
      jung = to_special_token(jung)
      jong = to_special_token(jong)
      decomposed_token = decomposed_token + cho + jung + jong

    # 만약 char(음절)이 한글이 아닐 경우 자모를 나누지 않고 추가
    except Exception as exception:
      if type(exception).__name__ == 'NotHangulException':
        decomposed_token += char
    
  # 단어 토큰의 자모 단위 분리 결과를 추가
  return decomposed_token

## 티어 예측

In [4]:
data_word_to_index = {'-': 1, 'ㅇ': 2, 'ㅏ': 3, 'ㄴ': 4, 'ㅣ': 5, 'ㄱ': 6, 'ㄹ': 7, 'ㅁ': 8, 'ㅗ': 9, ' ': 10, 'ㅅ': 11, 'ㅈ': 12, 'ㅓ': 13, 'ㅡ': 14, 'ㅜ': 15, 'ㅂ': 16, 'ㄷ': 17, 'ㅎ': 18, 'ㅐ': 19, 'a': 20, 'ㅕ': 21, 'ㅔ': 22, 'e': 23, 'i': 24, 'o': 25, 'ㅊ': 26, 'n': 27, 'ㅌ': 28, 'l': 29, 'ㅍ': 30, 'ㅋ': 31, 'r': 32, 's': 33, 'u': 34, '1': 35, 't': 36, 'ㅘ': 37, 'ㄲ': 38, 'h': 39, 'd': 40, 'ㅑ': 41, 'ㅛ': 42, 'I': 43, 'ㅠ': 44, 'g': 45, 'm': 46, 'y': 47, 'ㄸ': 48, 'k': 49, 'ㅝ': 50, '2': 51, '0': 52, 'c': 53, 'ㅆ': 54, 'ㅃ': 55, 'ㅉ': 56, 'ㅢ': 57, 'S': 58, 'A': 59, 'p': 60, 'w': 61, 'ㅟ': 62, 'T': 63, 'b': 64, 'O': 65, 'D': 66, 'E': 67, 'N': 68, 'G': 69, 'z': 70, 'f': 71, 'L': 72, 'R': 73, 'K': 74, 'M': 75, 'v': 76, '3': 77, 'ㅚ': 78, 'P': 79, 'C': 80, 'ㅖ': 81, 'x': 82, 'H': 83, '9': 84, 'j': 85, 'B': 86, '7': 87, '4': 88, '5': 89, '8': 90, 'J': 91, 'F': 92, '6': 93, 'Y': 94, 'W': 95, 'U': 96, 'q': 97, 'V': 98, 'X': 99, 'ㅄ': 100, 'Z': 101, 'ㅙ': 102, 'ㄺ': 103, 'ㄶ': 104, 'Q': 105, 'ㅞ': 106, 'ㅀ': 107, 'ㄻ': 108, 'ㅒ': 109, 'ㄼ': 110, 'ㄾ': 111, 'ㄵ': 112, 'ㄳ': 113, 'OOV': 114}

모델 불러오기

In [19]:
urllib.request.urlretrieve("https://github.com/oneonlee/Prediction-of-LoL-Tier-with-Summoner-Name/blob/main/model/jamo_best_model_biGRU_64.h5?raw=true", filename="jamo_best_model_biGRU_64.h5")

('jamo_best_model_biGRU_64.h5', <http.client.HTTPMessage at 0x7f33d507ca90>)

In [20]:
model = load_model('jamo_best_model_biGRU_64.h5')

In [21]:
def tier_predict(nickname):
    jamo_nickname = word_to_jamo(nickname)
    encoded_nickname = []
    for jamo in jamo_nickname:
        try:
            # 단어 집합에 있는 단어라면 해당 단어의 정수를 리턴.
            encoded_nickname.append(data_word_to_index[jamo])
        except KeyError:
            # 만약 단어 집합에 없는 단어라면 'OOV'의 정수를 리턴.
            encoded_nickname.append(data_word_to_index['OOV'])
    padded_nickname = pad_sequences([encoded_nickname], maxlen=16)
    
    tier_index = np.argmax(model.predict(padded_nickname))
    tier_confidence = np.max(model.predict(padded_nickname))
    tier_list = ['마스터 이상', '다이아몬드', '플래티넘', '골드', '실버', '브론즈', '아이언']

    return tier_list[tier_index], tier_confidence

In [22]:
tier_predict('KT way')

('마스터 이상', 0.25377932)

In [23]:
tier_predict('플래티넘 문지기')

('플래티넘', 0.215069)

In [24]:
tier_predict('아몬드가 죽으면')

('다이아몬드', 0.28879693)

In [25]:
tier_predict('아이언맨')

('아이언', 0.18356499)

In [26]:
tier_predict('정보통신기술협회')

('플래티넘', 0.18466832)